## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.15,-70.92,50.00,87,100,4.70,moderate rain
1,1,Mongo,TD,12.18,18.69,73.53,25,0,4.23,clear sky
2,2,Nikolskoye,RU,59.70,30.79,23.00,92,75,4.47,broken clouds
3,3,Mar Del Plata,AR,-38.00,-57.56,66.99,93,75,9.17,broken clouds
4,4,Beya,RU,53.05,90.91,-10.05,84,96,2.48,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Kavieng,PG,-2.57,150.80,85.62,66,14,9.86,few clouds
10,10,Port Blair,IN,11.67,92.75,83.57,71,47,10.83,light rain
13,13,Omboue,GA,-1.57,9.26,78.06,87,100,4.70,overcast clouds
15,15,Gizo,SB,-8.10,156.84,81.19,77,100,7.81,overcast clouds
16,16,Busselton,AU,-33.65,115.33,79.00,34,0,4.00,clear sky
22,22,Avarua,CK,-21.21,-159.78,77.00,83,54,1.12,broken clouds
25,25,Puerto Madero,MX,14.72,-92.42,75.07,75,0,6.13,clear sky
28,28,Carnarvon,AU,-24.87,113.63,84.20,66,0,18.34,clear sky
29,29,New Norfolk,AU,-42.78,147.06,75.20,41,75,20.80,light rain
32,32,Salalah,OM,17.02,54.09,77.00,29,0,14.99,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                181
City                   181
Country                180
Lat                    181
Lng                    181
Max Temp               181
Humidity               181
Cloudiness             181
Wind Speed             181
Current Description    181
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
clean_df.count()

City_ID                180
City                   180
Country                180
Lat                    180
Lng                    180
Max Temp               180
Humidity               180
Cloudiness             180
Wind Speed             180
Current Description    180
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Kavieng,PG,85.62,few clouds,-2.57,150.80,
10,Port Blair,IN,83.57,light rain,11.67,92.75,
13,Omboue,GA,78.06,overcast clouds,-1.57,9.26,
15,Gizo,SB,81.19,overcast clouds,-8.10,156.84,
16,Busselton,AU,79.00,clear sky,-33.65,115.33,
22,Avarua,CK,77.00,broken clouds,-21.21,-159.78,
25,Puerto Madero,MX,75.07,clear sky,14.72,-92.42,
28,Carnarvon,AU,84.20,clear sky,-24.87,113.63,
29,New Norfolk,AU,75.20,light rain,-42.78,147.06,
32,Salalah,OM,77.00,clear sky,17.02,54.09,


In [13]:
len(hotel_df)

180

In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping.")
        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.


In [25]:
#hotel_df
#hotel_df.count()

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [16]:
import numpy as np
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [18]:
#len(clean_hotel_df)

174

In [19]:
#len(clean_hotel_df)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Kavieng,PG,85.62,few clouds,-2.57,150.80,Nusa Island Retreat
10,Port Blair,IN,83.57,light rain,11.67,92.75,Welcomhotel Bay Island Port Blair
13,Omboue,GA,78.06,overcast clouds,-1.57,9.26,Hotel Olako
15,Gizo,SB,81.19,overcast clouds,-8.10,156.84,Imagination Island
16,Busselton,AU,79.00,clear sky,-33.65,115.33,Observatory Guest House
...,...,...,...,...,...,...,...
671,Bentiu,SS,85.26,clear sky,9.23,29.83,Mandela hôtel
672,Cuamba,MZ,81.75,broken clouds,-14.80,36.54,Hotel Vision
689,Hobyo,SO,81.12,few clouds,5.35,48.53,Halane Hotel and Restaurent
693,Maroantsetra,MG,87.44,overcast clouds,-15.43,49.73,La Pagode


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and
{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))